In [25]:
import numpy as np
import matplotlib as mpl
import os
import matplotlib.pyplot as plt
import matplotlib.animation
import tensorflow as tf
from numpy import genfromtxt
import pandas as pd
import sys
import math 

sys.path.append(os.getcwd())
import tsa_utils
import deep_cnn
import mini_cnn

from shutil import copytree


In [17]:
#Model parameters
BATCH_SIZE=10
FILTER_COUNT=24
KERNEL_SIZE1=(1,6,6)
DEPTHSTRIDE=1
XSTRIDE=1
YSTRIDE=1
POOLSIZE1=(16,3,3)
POOLSIZE2=(3,3)
POOL_STRIDES1=(1,2,2)
POOL_STRIDES=(1,1)
STEPS=3000
XSIZE=392
#XSIZE=512
SIZE=660
#XSIZE=270
YSIZE=340
LEARNING_RATE=0.001
IMAGE_DEPTH=16
CHANNELS=1
WEIGHTS=[1, 5]
FLAT_POOL_SIZE=43680

In [18]:
DATA_PATH="/data_volume/"
CHECKPOINT_PATH="/output/new_output2"
MODEL_ID="tsa_nb"

In [19]:
import numpy as np
import matplotlib as mpl
import os
import matplotlib.pyplot as plt
import matplotlib.animation
import tensorflow as tf
from numpy import genfromtxt
import pandas as pd
import sys
sys.path.append(os.getcwd())
import tsa_utils

tf.logging.set_verbosity(tf.logging.INFO)

class ZoneModel():

    def __init__(self, model_id, ids, zone, x_slice, y_slice, data_path, labels, checkpoint_path=".", localize=False):
        self.model_id = model_id
        self.ids = ids
        self.zone = zone if zone else ""
        self.x_slice = x_slice
        self.y_slice = y_slice
        self.data_path = data_path
        self.checkpoint_path = checkpoint_path
        self.labels = labels
        self.localize = localize

    def build_model(self, data, labels, mode):
        if mode == tf.contrib.learn.ModeKeys.INFER:
            BATCH_SIZE=1
        else:
            BATCH_SIZE=10
        print(XSIZE, YSIZE)
        data = tf.reshape(data, [BATCH_SIZE, IMAGE_DEPTH, YSIZE, XSIZE, CHANNELS])
        conv1 = tf.layers.conv3d(inputs=data, filters=FILTER_COUNT, kernel_size=KERNEL_SIZE1, padding="same", 
                strides=(DEPTHSTRIDE,XSTRIDE,YSTRIDE), name="conv1", trainable=not self.localize)
        print(conv1)
        conv2 = tf.layers.conv3d(inputs=conv1, filters=FILTER_COUNT, kernel_size=(3,3,3), padding="same",
                strides=(1, 2, 1), name="conv2", activation=tf.nn.relu, trainable=not self.localize)
        print(conv2)
        pool1 = tf.layers.max_pooling3d(inputs=conv2, pool_size=POOLSIZE1, strides=POOL_STRIDES1, name="pool1")
        print(pool1)
        flattener = tf.reshape(pool1, [BATCH_SIZE, 84, 195, 24])
        pool2 = tf.layers.max_pooling2d(inputs=flattener, pool_size=(3,3), strides=(3,3), name="pool2")
        print(pool2)
        conv3 = tf.layers.conv2d(inputs=pool2, filters=FILTER_COUNT, kernel_size=(3,3), padding="same", 
                strides=(XSTRIDE,YSTRIDE), name="conv3", trainable=not self.localize)
        print(conv3)
        pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=(2,2), strides=POOL_STRIDES, name="pool1")
        print(pool3)
        conv4 = tf.layers.conv2d(inputs=pool3, filters=FILTER_COUNT, kernel_size=(3,3), padding="same", 
                strides=(1, 1), name="conv4", activation=tf.nn.relu, trainable=not self.localize)
        print(conv4)
        pool4 = tf.layers.max_pooling2d(inputs=conv4, pool_size=POOLSIZE2, strides=POOL_STRIDES, name="pool2", padding="same")
        print(pool4)
        flat_pool = tf.reshape(pool2, [BATCH_SIZE, FLAT_POOL_SIZE])
        flat_pool=tf.identity(flat_pool, name="flat_pool")

        logits = tf.layers.dense(inputs=flat_pool, units=2, trainable=True)
        logits = tf.identity(logits, name="logits")
        logits = tf.reshape(logits, [BATCH_SIZE,2])
        predictions = {
            "classes": tf.argmax(
              input=logits, axis=1, name="classes"),
          "probabilities": tf.nn.softmax(
              logits, name="softmax_tensor")}

        if mode == tf.contrib.learn.ModeKeys.TRAIN:
            flat_labels = tf.reshape(labels, [BATCH_SIZE, 2])
            test_labels=tf.identity(flat_labels, name="labels")
            class_weights=tf.reduce_sum(tf.multiply(flat_labels, tf.constant(WEIGHTS, dtype=tf.int64)), axis=1)
            loss = tf.losses.softmax_cross_entropy(onehot_labels=test_labels, logits=logits, weights=class_weights)
            train_op = tf.contrib.layers.optimize_loss(
                loss=loss,
                global_step=tf.contrib.framework.get_global_step(),
                learning_rate=LEARNING_RATE,
                optimizer="SGD")
        if mode == tf.contrib.learn.ModeKeys.INFER:
            return tf.contrib.learn.ModelFnOps(mode=mode, predictions=predictions)
        return tf.contrib.learn.ModelFnOps(mode=mode, predictions=predictions, loss=loss, train_op=train_op)

    def train_model(self, tensors_to_log, reuse=False):
        if reuse:
            tsa_classifier = self.model
        else:
            tsa_classifier = tf.contrib.learn.Estimator(model_fn=self.build_model, 
                                                     model_dir=self.checkpoint_path + "/" + self.model_id + self.zone)
        logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=100)
        tsa_classifier.fit(
            x=InputImagesIterator(self.ids, self.data_path, 10000, self.y_slice, self.x_slice), 
            y=InputLabelsIterator(self.ids, self.labels), 
            steps=STEPS, 
            batch_size=BATCH_SIZE, 
            monitors=[logging_hook])
        self.model = tsa_classifier

    def load_model(self):
        tsa_classifier = tf.contrib.learn.Estimator(model_fn=self.build_model, 
                                                    model_dir=self.checkpoint_path + "/" + self.model_id + self.zone)
        print ("LOADED MODEL AT STEP " + str(tsa_classifier.get_variable_value("global_step")) + " FROM " + self.checkpoint_path + "/" + self.model_id + self.zone)
        self.model = tsa_classifier

    def bootstrap_model(self, path=None):
        if path:
            checkpoint_path = path
        else:
            checkpoint_path = self.checkpoint_path + "/" + self.model_id + self.zone
        print ("USING CHECKPOINT PATH  " + checkpoint_path)
        tsa_classifier = tf.contrib.learn.Estimator(model_fn=self.build_model, 
                                                    model_dir=checkpoint_path)
        print ("BOOTSTRAPPED AT STEP " + str(tsa_classifier.get_variable_value("global_step")))
        self.model = tsa_classifier 

    def predict(self):
        return self.model.predict(x=InputImagesIterator(self.ids, 
                                                            self.data_path, 
                                                            10000, 
                                                            self.y_slice,
                                                            self.x_slice, False))

In [20]:
image_df = pd.read_csv(DATA_PATH + '/stage1_labels.csv')
image_df['zone'] = image_df['Id'].str.split("_", expand=True)[1].str.strip()
image_df['id'] = image_df['Id'].str.split("_", expand=True)[0].str.strip()

ids = image_df["id"].unique()
ids.sort()
training_ids = ids

labels = image_df[image_df['id'].isin(training_ids)]
labels = labels.sort_values("id")

tensors_to_log =  {"probabilities": "softmax_tensor",
                    "actual":"labels"}


In [21]:
slice_locations = {1:("top","right"),
	2:("middle","right"), 
	3:("top","left"),
	4:("middle","left"),
	5:("top","middle"),
	6:("top","right"),
	7:("top","left"),
	8:("middle","right"),
	9:("middle","middle"),
	10:("middle","left"),
	11:("bottom","right"),
	12:("bottom","left"),
	13:("bottom","right"),
	14:("bottom","left"),
	15:("bottom","right"),
	16:("bottom","left"),
	17:("top","middle")}

In [26]:
with tf.Session() as sess:
	for zone in range(2,5):
		print("FITTING MODEL FOR ZONE " + str(zone))
		train_labels = labels[labels["zone"]=="Zone"+str(zone)]
		train_labels["class0"] = 0
		train_labels["class1"] = 0
		train_labels.loc[train_labels['Probability'] == 0, 'class0'] = 1
		train_labels.loc[train_labels['Probability'] == 1, 'class1'] = 1
		train_labels = np.reshape(np.array(train_labels[["class0","class1"]]), [-1,2])
        #model = ZoneModel(MODEL_ID, training_ids, "Zone" + str(zone), slice_locations[zone][1], slice_locations[zone][0], DATA_PATH, train_labels, CHECKPOINT_PATH, localize=False)
        model = ZoneModel(MODEL_ID, training_ids, "Zone" + str(zone), "trimmed", slice_locations[zone][0], DATA_PATH, train_labels, CHECKPOINT_PATH, localize=False)     
        model.train_model(tensors_to_log, reuse=False)

FITTING MODEL FOR ZONE 2


/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


FITTING MODEL FOR ZONE 3
FITTING MODEL FOR ZONE 4
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa1f5d2fd90>, '_model_dir': '/output/new_output2/tsa_nbZone4', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_session_config': None, '_tf_random_seed': None, '_environment': 'local', '_num_worker_replicas': 0, '_task_id': 0, '_save_summary_steps': 100, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_evaluation_master': '', '_master': ''}
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompa

INFO:tensorflow:Saving checkpoints for 1028 into /output/new_output2/tsa_nbZone4/model.ckpt.
INFO:tensorflow:global_step/sec: 0.189477
INFO:tensorflow:loss = 0.960438, step = 1116 (527.768 sec)
INFO:tensorflow:probabilities = [[ 0.33523101  0.66476899]
 [ 0.80959141  0.19040857]
 [ 0.97864771  0.02135229]
 [ 0.58162558  0.41837436]
 [ 0.97054905  0.02945091]
 [ 0.55545074  0.44454932]
 [ 0.82198948  0.17801055]
 [ 0.66451621  0.33548379]
 [ 0.6918084   0.3081916 ]
 [ 0.86518633  0.13481364]], actual = [[0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]] (527.768 sec)
INFO:tensorflow:Saving checkpoints for 1142 into /output/new_output2/tsa_nbZone4/model.ckpt.
INFO:tensorflow:global_step/sec: 0.188846
INFO:tensorflow:loss = 0.408141, step = 1216 (529.531 sec)
INFO:tensorflow:probabilities = [[ 0.32911074  0.67088926]
 [ 0.77487344  0.22512649]
 [ 0.66304928  0.33695069]
 [ 0.73330486  0.26669511]
 [ 0.53353071  0.46646932]
 [ 0.71910316  0.28089681]
 [ 0.76321214  0.236

INFO:tensorflow:Saving checkpoints for 2510 into /output/new_output2/tsa_nbZone4/model.ckpt.
INFO:tensorflow:global_step/sec: 0.18925
INFO:tensorflow:loss = 0.0722558, step = 2516 (528.401 sec)
INFO:tensorflow:probabilities = [[ 0.98588264  0.0141174 ]
 [ 0.95421708  0.04578298]
 [ 0.0406707   0.95932937]
 [ 0.97274065  0.0272594 ]
 [ 0.99557883  0.00442114]
 [ 0.99464983  0.00535024]
 [ 0.99987352  0.00012648]
 [ 0.98452282  0.01547713]
 [ 0.99363774  0.00636225]
 [ 0.07583636  0.92416358]], actual = [[1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]] (528.401 sec)
INFO:tensorflow:global_step/sec: 0.189457
INFO:tensorflow:loss = 0.0926149, step = 2616 (527.824 sec)
INFO:tensorflow:probabilities = [[ 0.99851984  0.00148018]
 [ 0.99498111  0.00501887]
 [ 0.99757993  0.00241998]
 [ 0.99997663  0.00002332]
 [ 0.96920645  0.03079357]
 [ 0.99105936  0.00894058]
 [ 0.87877804  0.12122196]
 [ 0.99622774  0.00377229]
 [ 0.9827404   0.01725966]
 [ 0.48358932  0.51641065]], ac

In [ ]:
# copy in data from pervious model if needed 
from shutil import copytree

for zone in range(1,18):
	copytree("/models/tsa_nbZone" + str(zone), "/output/tsa_nbZone" +str(zone))

In [27]:
import numpy as np
import matplotlib as mpl
import os
import matplotlib.pyplot as plt
import matplotlib.animation
import tensorflow as tf
from numpy import genfromtxt
import pandas as pd
import sys
import math 
import sklearn
from sklearn import metrics

sys.path.append(os.getcwd())
import tsa_utils
import mini_cnn

image_df = pd.read_csv(DATA_PATH + 'stage1_sample_submission.csv')
image_df['zone'] = image_df['Id'].str.split("_", expand=True)[1].str.strip()
image_df['id'] = image_df['Id'].str.split("_", expand=True)[0].str.strip()

ids = image_df["id"].unique()
ids.sort()
labels = image_df[image_df['id'].isin(ids)]
all_labels = labels.sort_values("id")

csv_file = open(CHECKPOINT_PATH + "/submission.csv", "w+")
csv_file.write("Id,Probability\n")
for zone in range(4,5):
	print("EVALUATING MODEL FOR ZONE " + str(zone))
	labels = all_labels[all_labels["zone"]=="Zone"+str(zone)]
	labels["class0"] = 0
	labels["class1"] = 0
	labels.loc[labels['Probability'] == 0, 'class0'] = 1
	labels.loc[labels['Probability'] == 1, 'class1'] = 1
	labels = np.reshape(np.array(labels[["class0","class1"]]), [-1,2])

	model = ZoneModel(MODEL_ID, ids, "Zone" + str(zone), slice_locations[zone][1], slice_locations[zone][0], DATA_PATH, image_df)
	#model = ZoneModel(MODEL_ID, ids, "Zone" + str(zone), "both", "both", DATA_PATH, labels, checkpoint_path=CHECKPOINT_PATH)
	model.load_model()
	print ("LOADED MODEL " + str(model))
	predicted = np.array([x["probabilities"][1] for x in list(model.predict())])
	for i, prediction in enumerate(predicted):
		csv_file.write(str(ids[i]) + "_Zone" + str(zone) + "," + str(prediction) + "\n")
csv_file.close()


EVALUATING MODEL FOR ZONE 4


/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa1dfc27f10>, '_model_dir': './tsa_nbZone4', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_session_config': None, '_tf_random_seed': None, '_environment': 'local', '_num_worker_replicas': 0, '_task_id': 0, '_save_summary_steps': 100, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_evaluation_master': '', '_master': ''}
LOADED MODEL AT STEP 531 FROM ./tsa_nbZone4
LOADED MODEL <__main__.ZoneModel instance at 0x7fa1dfe0e638>
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  

InvalidArgumentError: Assign requires shapes of both tensors to match. lhs shape= [43680,2] rhs shape= [59136,2]
	 [[Node: save/Assign_9 = Assign[T=DT_FLOAT, _class=["loc:@dense/kernel"], use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/gpu:0"](dense/kernel, save/RestoreV2_9/_3)]]

Caused by op u'save/Assign_9', defined at:
  File "/usr/local/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/local/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/lib/python2.7/site-packages/zmq/eve

In [1]:
import os 
import numpy as np

def read_header(infile):
    """Read image header (first 512 bytes)
    """
    h = dict()
    fid = open(infile, 'rb')
    h['filename'] = b''.join(np.fromfile(fid, dtype = 'S1', count = 20))
    h['parent_filename'] = b''.join(np.fromfile(fid, dtype = 'S1', count = 20))
    h['comments1'] = b''.join(np.fromfile(fid, dtype = 'S1', count = 80))
    h['comments2'] = b''.join(np.fromfile(fid, dtype = 'S1', count = 80))
    h['energy_type'] = np.fromfile(fid, dtype = np.int16, count = 1)
    h['config_type'] = np.fromfile(fid, dtype = np.int16, count = 1)
    h['file_type'] = np.fromfile(fid, dtype = np.int16, count = 1)
    h['trans_type'] = np.fromfile(fid, dtype = np.int16, count = 1)
    h['scan_type'] = np.fromfile(fid, dtype = np.int16, count = 1)
    h['data_type'] = np.fromfile(fid, dtype = np.int16, count = 1)
    h['date_modified'] = b''.join(np.fromfile(fid, dtype = 'S1', count = 16))
    h['frequency'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['mat_velocity'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['num_pts'] = np.fromfile(fid, dtype = np.int32, count = 1)
    h['num_polarization_channels'] = np.fromfile(fid, dtype = np.int16, count = 1)
    h['spare00'] = np.fromfile(fid, dtype = np.int16, count = 1)
    h['adc_min_voltage'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['adc_max_voltage'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['band_width'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['spare01'] = np.fromfile(fid, dtype = np.int16, count = 5)
    h['polarization_type'] = np.fromfile(fid, dtype = np.int16, count = 4)
    h['record_header_size'] = np.fromfile(fid, dtype = np.int16, count = 1)
    h['word_type'] = np.fromfile(fid, dtype = np.int16, count = 1)
    h['word_precision'] = np.fromfile(fid, dtype = np.int16, count = 1)
    h['min_data_value'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['max_data_value'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['avg_data_value'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['data_scale_factor'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['data_units'] = np.fromfile(fid, dtype = np.int16, count = 1)
    h['surf_removal'] = np.fromfile(fid, dtype = np.uint16, count = 1)
    h['edge_weighting'] = np.fromfile(fid, dtype = np.uint16, count = 1)
    h['x_units'] = np.fromfile(fid, dtype = np.uint16, count = 1)
    h['y_units'] = np.fromfile(fid, dtype = np.uint16, count = 1)
    h['z_units'] = np.fromfile(fid, dtype = np.uint16, count = 1)
    h['t_units'] = np.fromfile(fid, dtype = np.uint16, count = 1)
    h['spare02'] = np.fromfile(fid, dtype = np.int16, count = 1)
    h['x_return_speed'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['y_return_speed'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['z_return_speed'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['scan_orientation'] = np.fromfile(fid, dtype = np.int16, count = 1)
    h['scan_direction'] = np.fromfile(fid, dtype = np.int16, count = 1)
    h['data_storage_order'] = np.fromfile(fid, dtype = np.int16, count = 1)
    h['scanner_type'] = np.fromfile(fid, dtype = np.int16, count = 1)
    h['x_inc'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['y_inc'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['z_inc'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['t_inc'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['num_x_pts'] = np.fromfile(fid, dtype = np.int32, count = 1)
    h['num_y_pts'] = np.fromfile(fid, dtype = np.int32, count = 1)
    h['num_z_pts'] = np.fromfile(fid, dtype = np.int32, count = 1)
    h['num_t_pts'] = np.fromfile(fid, dtype = np.int32, count = 1)
    h['x_speed'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['y_speed'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['z_speed'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['x_acc'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['y_acc'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['z_acc'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['x_motor_res'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['y_motor_res'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['z_motor_res'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['x_encoder_res'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['y_encoder_res'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['z_encoder_res'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['date_processed'] = b''.join(np.fromfile(fid, dtype = 'S1', count = 8))
    h['time_processed'] = b''.join(np.fromfile(fid, dtype = 'S1', count = 8))
    h['depth_recon'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['x_max_travel'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['y_max_travel'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['elevation_offset_angle'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['roll_offset_angle'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['z_max_travel'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['azimuth_offset_angle'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['adc_type'] = np.fromfile(fid, dtype = np.int16, count = 1)
    h['spare06'] = np.fromfile(fid, dtype = np.int16, count = 1)
    h['scanner_radius'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['x_offset'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['y_offset'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['z_offset'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['t_delay'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['range_gate_start'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['range_gate_end'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['ahis_software_version'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['spare_end'] = np.fromfile(fid, dtype = np.float32, count = 10)
    return h


def read_data(infile, vertical="both", horizontal="both"):
    """Read any of the 4 types of image files, returns a numpy array of the image contents
    """
    extension = os.path.splitext(infile)[1]
    h = read_header(infile)
    nx = int(h['num_x_pts'])
    ny = int(h['num_y_pts'])
    nt = int(h['num_t_pts'])
    fid = open(infile, 'rb')
    fid.seek(512) #skip header
    if extension == '.aps' or extension == '.a3daps':
        if(h['word_type']==7): #float32
            data = np.fromfile(fid, dtype = np.float32, count = nx * ny * nt)
        elif(h['word_type']==4): #uint16
            data = np.fromfile(fid, dtype = np.uint16, count = nx * ny * nt)
        data = data * h['data_scale_factor'] #scaling factor
        data = data.reshape(nx, ny, nt, order='F').copy() #make N-d image
        if vertical == "bottom":
            data = data[:, :340, :] 
        elif vertical == "top":
            data = data[:, 320:660, :] 
        elif vertical == "middle":
            data = data[:, 170:510, :] 
        rotated_data = []
        if horizontal == "right":
            for i in range(0,16):
                increment = 10
                rotated_data.append(data[(i * increment) + 50:(i * increment) + 320,:,i])
        elif horizontal == "left":
            for i in range(0,16):
                increment = 10
                rotated_data.append(data[210 - (i * increment): 480-(i * increment):,:,i])
        elif horizontal == "middle":
            for i in range(0,16):
                increment = 10
                rotated_data.append(data[130:400,:,i])
        elif horizontal == "trimmed":
            for i in range(0,16):
                rotated_data.append(data[60:-60:,:,i])
        else:
            for i in range(0,16):
                rotated_data.append(data[:,:,i])
        data = np.array(rotated_data)  
    elif extension == '.a3d':
        if(h['word_type']==7): #float32
            data = np.fromfile(fid, dtype = np.float32, count = nx * ny * nt)
        elif(h['word_type']==4): #uint16
            data = np.fromfile(fid, dtype = np.uint16, count = nx * ny * nt)
        data = data * h['data_scale_factor'] #scaling factor
        data = data.reshape(nx, nt, ny, order='F').copy() #make N-d image
    elif extension == '.ahi':
        data = np.fromfile(fid, dtype = np.float32, count = 2* nx * ny * nt)
        data = data.reshape(2, ny, nx, nt, order='F').copy()
        real = data[0,:,:,:].copy()
        imag = data[1,:,:,:].copy()
    fid.close()
    if extension != '.ahi':
        return np.swapaxes(data, 2, 1)
    else:
        return real, imag

class InputImagesIterator:
    def __init__(self, ids, data_path, contrast=1, vertical="both", horizontal="both", repeating=True):
        self.ids=ids
        self.contrast = contrast
        self.data_path=data_path
        self.i = -1
        self.vertical = vertical
        self.horizontal = horizontal
        self.repeating = repeating

    def __iter__(self):
        return self

    def next(self):
        # print ("id " + str(self.ids[self.i-1])) 
        # print("image iter " + str(self.i))
        if self.i < len(self.ids) -1:
            self.i = self.i + 1
            #print("IMAGES ITERATOR " + str(self.ids[self.i - 1]))
            return np.stack(read_data(self.data_path + self.ids[self.i] + ".aps", self.vertical, self.horizontal) * self.contrast)
        else:
            if not self.repeating:
                raise StopIteration()
            #Restart iteration, cycle back through
            self.i = -1
            return np.stack(read_data(self.data_path + self.ids[0] + ".aps", self.vertical, self.horizontal) * self.contrast)

class InputLabelsIterator:
    def __init__(self, ids, labels):
        self.ids=ids
        self.labels=labels
        self.i=-1

    def __iter__(self):
        return self

    def next(self):
        #print("in label iter " + str(self.ids[self.i -1])) 
        #print("in label iter " + str(self.test_labels[self.i -1])) 
        #print("label iter " + str(self.i))
        if self.i < len(self.ids) -1:
            self.i = self.i + 1
            #print("LABEL" + str(self.test_labels[self.i -1]))
            return(self.labels[self.i])
        else:
            #Restart iteration, cycle back through
            self.i = -1
            #raise StopIteration()
            return(self.labels[0])
